In [6]:
"""
Реализовать самостоятельно логистическую регрессию
Обучить ее методом градиентного спуска
Методом nesterov momentum
Методом rmsprop
В качестве dataset’а взять Iris, оставив 2 класса:
Iris Versicolor
Iris Virginica
"""

import pandas as pd
import numpy as np
import math 
from sklearn.datasets import load_iris
import tensorflow as tf

N = 100
X = np.random.uniform(low=0, high=100, size=N)
Y = 2*X + 1 + np.random.normal(scale=5, size=N)

In [7]:
#Применим метод градиентного спуска для случаного уравнения. Наша задача - получить наиболее оптимальные коэффециенты params
EPOCHS = 20
LEARNING_RATE = 0.0001

In [8]:
costs = []   # потери 
params = []
preds = []
params = np.random.normal(size=(2,))  # случайно заданная строка 2-х параметров (исходные точки)

for i in range(EPOCHS):
    predictions = params[0] + params[1] * X
    preds.append(predictions)

    cost = np.sum(np.square(predictions - Y)) / (2 * len(predictions))
    costs.append(cost)
    
    params[0] -= LEARNING_RATE * np.sum(predictions - Y) / len(predictions)
    params[1] -= LEARNING_RATE * np.sum((predictions - Y) * X) / len(predictions)  
    print(params[0], params[1])


-1.8150215743611149 -0.28372478288756975
-1.8039091720835734 0.44742984331796043
-1.7962700635340292 0.9493522826988267
-1.7910152999352102 1.2939115442518956
-1.7873973403659926 1.5304442233308464
-1.784903013371097 1.6928188771214165
-1.78318003730938 1.8042855729949054
-1.7819865780854112 1.8808050036317088
-1.7811566215704846 1.9333338560284623
-1.7805762024824032 1.9693936702803347
-1.7801670860007455 1.9941478288423138
-1.7798755655316376 2.0111408874887684
-1.7796647726038517 2.0228061107990976
-1.7795093977707674 2.0308138848217228
-1.7793920666182306 2.036310895196333
-1.779300851968544 2.0400843180116595
-1.7792275660597145 2.042674533449703
-1.7791665881616587 2.0444525013824375
-1.779114059758861 2.0456728781262865
-1.7790673320598338 2.0465104800007303


In [9]:
# метод rmsprop в TensorFlow (https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/RMSprop?hl=ru)

tf.keras.optimizers.RMSprop(
    learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop', **kwargs) 

# метод nesterov momentum в TensorFlow (https://www.tensorflow.org/api_docs/python/tf/compat/v1/train/MomentumOptimizer?hl=ru)
tf.compat.v1.train.MomentumOptimizer(
    learning_rate, momentum, use_locking=False, name='Momentum',
    use_nesterov=True
)

NameError: name 'kwargs' is not defined

In [10]:
# Реализуем код градиентного спуска в явном виде для датасета Iris

X,y = load_iris(return_X_y=True)
filter = y != 2              # накладываем фильтр - только 2 признака
X = X[filter]
y = y[filter]

COEFS = np.random.randn(5)   # генерируем случайные коэффициенты

def predict_proba(coefs, x):
  # формула логистической регрессии:
  return 1. / (1. + np.exp( -(x.dot(coefs[:4]) + coefs[-1]) ) ) 

# теперь на основе модели предсказываем класс (но модель пока не обучена: 
# это сделаем градиентным спуском чуть позже)
def predict_class(coefs, x):
    probas = predict_proba(coefs, x)
    return (probas > 0.5).astype(np.float)

# явно прописываем функцию потерь на основе ее формулы
def bce_loss(coefs, x, y):
    probas = predict_proba(coefs, x)
    filter_ones = y == 1
    loss = -1. * (np.sum(np.log(probas[filter_ones])) + np.sum(np.log(1. - probas[~filter_ones]))) / len(y)
    return loss

# расчет градиента:
# он зависит от двух сущностей: от модели и функции потерь
def grad(coefs, x, y):
    probas = predict_proba(coefs, x)
    delta = probas - y
    modified_x = x.T * delta
    deltas = np.mean(modified_x, axis=1)
    return deltas, np.mean(delta)

# обучение модели методом градиентного спуска
def learn_sgd(coefs, x, y, num_epochs=20, learning_rate=0.0001):
    losses = []                                               # solutions = list()
    for e in range(num_epochs):
        grad_coefs, grad_bias = grad(coefs, x, y)             # gradient = derivative(projected[0], projected[1])
        coefs[:-1] = coefs[:-1] - learning_rate * grad_coefs
        coefs[-1] = coefs[-1] - learning_rate * grad_bias
        loss = bce_loss(coefs, x, y)
        losses.append(loss)
    return losses, coefs

grad(COEFS, X, y)

(array([-2.92649172, -1.36006538, -2.11183437, -0.65850995]),
 -0.49200241340550505)

In [11]:
# Реализуем метод Нестеров моментум для того же самого датасета. Эвристика заключается в добавлении импульса (momentum) 
# и расчете градиента для следующей точки графика. Оказавшись в одном локальном минимуме, 
# мы сможем перескочиться через следующую "горку", чтобы увидеть там локальный минимум бОльшей глубины. 
# Вот общая формула: 
#     Nesterov Momentum v_t = momentum* v_t{t-1} + learning_rate*grad(coefs - momentum v_t{t-1}), 
#     где coefs = coefs{t-1} - v_t. 
# Видим, что скорость в точке t соотносится со скоростью в точке t-1.

def learn_nesterov(coefs, x, y, num_epochs=20, momentum=0.9, learning_rate=0.0001):
    v_t = [0 for _ in range(len(coefs))]
    losses =[]
    for i in range(num_epochs):
        pr_coefs  = [coefs[i] - momentum*v_t[i] for i in range(len(coefs))]
        gr_coefs = grad(pr_coefs, x, y)
        for i in range(len(coefs)):
            v_t[i]=momentum*v_t[i] + learning_rate*gr_coefs[i]
            coefs[i] -= v_t[i]
        losses.append(bce_loss(x, y, coefs))
    return losses, coefs

learn_nesterov(COEFS, X, y)

ValueError: setting an array element with a sequence.

In [ ]:
# Теперь займемся методом RMSprop (применим его для того же самого датасета Iris)

def learn_RMSprop(coefs, x, y, num_epochs=20, momentum=0.9, learning_rate=0.0001):
    e = 10**(-8)
    S = [0 for _ in range(len(coefs))]
    losses =[]
    for it in range(num_epochs):
        gr_coefs = grad(x, y, coefs)
        gr_coefs_2 = [x**2 for x in gr_coefs]
        for i in range(len(coefs)):
            S[i]=momentum*S[i] + (1-momentum)*gr_coefs_2[i]
            coefs[i] = coefs[i]-learning_rate* (gr_coefs[i]/(math.sqrt(S[i])+e))  
        losses.append(bce_loss(coefs, x, y))
    return losses, coefs

learn_RMSprop(COEFS, X, y)